In [ ]:
# Install necessary libraries
!pip install xgboost

# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load your dataset using the specified file path
file_path = '/content/drive/My Drive/fyp/heart.csv'  # Update this if the file name is different
df = pd.read_csv(file_path)



Mounted at /content/drive


In [ ]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:


# Check for missing values
print(df.isnull().sum())



# For example, fill missing values with the mean or drop the rows
df.fillna(df.mean(), inplace=True)

# Encode categorical variables if needed
df = pd.get_dummies(df, drop_first=True)

age        0
sex        0
cp         0
trtbps     0
chol       0
fbs        0
restecg    0
thalach    0
exang      0
ca         0
output     0
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'target' is your target column
X = df.drop('output', axis=1)  # Features
y = df['output']                # Target variable

# Split the data into training (70%) and testing (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# Initialize the models
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
xgboost = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
ridge = RidgeClassifier()

# Create a Voting Classifier
model = VotingClassifier(
    estimators=[
        ('logreg', logreg),
        ('dtree', dtree),
        ('xgboost', xgboost),
        ('ridge', ridge)
    ],
    voting='hard'  # Use 'hard' for majority voting
)

# Fit the Voting Classifier
model.fit(X_train, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:39:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.8387096774193549
Confusion Matrix:
 [[13  1]
 [ 4 13]]
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.93      0.84        14
           1       0.93      0.76      0.84        17

    accuracy                           0.84        31
   macro avg       0.85      0.85      0.84        31
weighted avg       0.85      0.84      0.84        31



In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Assume your model is already trained and named 'model'

# Create input widgets for the required attributes
age_widget = widgets.FloatText(description='Age:', placeholder='Enter age of the patient')
sex_widget = widgets.Dropdown(
    options=[('Male', 0), ('Female', 1)],
    description='Sex:'
)

# Labels for detailed descriptions
exang_label = widgets.Label(value='Exercise Induced Angina: (1 = Yes; 0 = No)')
exang_widget = widgets.Dropdown(
    options=[('No', 0), ('Yes', 1)],
    description='Induced Angina:'
)

ca_label = widgets.Label(value='Number of Major Vessels (0-3):')
ca_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=3,
    description='Major Vessels:',
    step=1
)

cp_label = widgets.Label(value='Chest Pain Type:')
cp_widget = widgets.Dropdown(
    options=[
        ('Typical Angina', 1),
        ('Atypical Angina', 2),
        ('Non-Anginal Pain', 3),
        ('Asymptomatic', 4)
    ],
    description='Chest Pain Type:'
)

# Detailed descriptions for BP, Cholesterol, and Max Heart Rate
trtbps_label = widgets.Label(value='Resting Blood Pressure (in mm Hg):')
trtbps_widget = widgets.FloatText(placeholder='Enter resting blood pressure')

chol_label = widgets.Label(value='Cholesterol (mg/dl):')
chol_widget = widgets.FloatText(placeholder='Enter cholesterol level')

thalach_label = widgets.Label(value='Maximum Heart Rate Achieved:')
thalach_widget = widgets.FloatText(placeholder='Enter max heart rate')

fbs_label = widgets.Label(value='Fasting Blood Sugar > 120 mg/dl: (1 = True; 0 = False)')
fbs_widget = widgets.Dropdown(
    options=[('False', 0), ('True', 1)],
    description='FBS:'
)

rest_ecg_label = widgets.Label(value='Resting ECG Results:')
rest_ecg_widget = widgets.Dropdown(
    options=[
        ('Normal', 0),
        ('ST-T Wave Abnormality', 1),
        ('Left Ventricular Hypertrophy', 2)
    ],
    description='ECG Results:'
)

# Create a button to trigger the prediction
predict_button = widgets.Button(description='Predict Risk')

# Define the prediction function
def on_predict_button_click(b):
    # Gather input data using correct feature names and order
    user_input = {
        'age': age_widget.value,
        'sex': sex_widget.value,
        'cp': cp_widget.value,               # Chest Pain Type
        'trtbps': trtbps_widget.value,       # Resting BP
        'chol': chol_widget.value,           # Cholesterol
        'fbs': fbs_widget.value,             # Fasting Blood Sugar
        'restecg': rest_ecg_widget.value,    # Resting ECG
        'thalach': thalach_widget.value,     # Max Heart Rate
        'exang': exang_widget.value,         # Exercise Induced Angina
        'ca': ca_widget.value                 # Number of Major Vessels
    }

    # Create DataFrame for input data, maintaining the correct order
    input_df = pd.DataFrame([user_input])

    # Ensure the columns are in the correct order
    input_df = input_df[['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'ca']]

    # Make predictions
    prediction = model.predict(input_df)

    # Display the result
    result = "High Risk" if prediction[0] == 1 else "Low Risk"
    print(result)

# Bind the button click event to the prediction function
predict_button.on_click(on_predict_button_click)

# Display the widgets and labels
display(
    age_widget,
    sex_widget,
    exang_label,
    exang_widget,
    ca_label,
    ca_widget,
    cp_label,
    cp_widget,
    trtbps_label,
    trtbps_widget,
    chol_label,
    chol_widget,
    fbs_label,
    fbs_widget,
    rest_ecg_label,
    rest_ecg_widget,
    thalach_label,
    thalach_widget,
    predict_button
)

FloatText(value=0.0, description='Age:')

Dropdown(description='Sex:', options=(('Male', 0), ('Female', 1)), value=0)

Label(value='Exercise Induced Angina: (1 = Yes; 0 = No)')

Dropdown(description='Induced Angina:', options=(('No', 0), ('Yes', 1)), value=0)

Label(value='Number of Major Vessels (0-3):')

...

Dropdown(description='ECG Results:', options=(('Normal', 0), ('ST-T Wave Abnormality', 1), ('Left Ventricular …

Label(value='Maximum Heart Rate Achieved:')

FloatText(value=0.0)

Button(description='Predict Risk', style=ButtonStyle())

High Risk
